Kelton McNair - 992018683
CS 601 - Machine Learning
Spring Semester Project 2025
Dr. Afshar
Random Forest Classifier
Dataset: Adult Or letter-recognition?

In [163]:
# pip install ucimlrepo

# from ucimlrepo import fetch_ucirepo 
  
# # fetch dataset 
# adult = fetch_ucirepo(id=2) 
  
# # data (as pandas dataframes) 
# X = adult.data.features 
# y = adult.data.targets 
  
# # metadata 
# print(adult.metadata) 
  
# # variable information 
# print(adult.variables) 



In [164]:
# from ucimlrepo import fetch_ucirepo 
  
# # fetch dataset 
# letter_recognition = fetch_ucirepo(id=59) 
  
# # data (as pandas dataframes) 
# X = letter_recognition.data.features 
# y = letter_recognition.data.targets 
  
# # metadata 
# print(letter_recognition.metadata) 
  
# # variable information 
# print(letter_recognition.variables) 


In [165]:
import numpy as np
import pandas as pd

#not sure what all I need for comparative evaluation sklearn model yet
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from numpy.random import default_rng

#hpyer parameters
label_col_name = "income"
num_trees_in_forest = 110
depth_threshold = 10 #32560 samples / 2 (10 times) = 32
# we could use a for loop to calc this based on sample size, and pass that to the two below 
split_leaf_minimum = 32
split_node_min_samples = (split_leaf_minimum * 2) + 1


In [166]:
data = pd.read_csv("adult/adult.data")


In [167]:
extracted_col_names = ["age", "workclass", "fnlwgt", "education", "education-num", "marital-status", "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss", "hours-per-week", "native-country", "income"] 

In [168]:
data.columns = extracted_col_names

In [169]:
def sample_information (num_samples):
    depth_counter = 0
    resulting = 15
    calcd_resulting = num_samples
    while (calcd_resulting > resulting):
        depth_counter+=1
        calcd_resulting = np.ceil(calcd_resulting / 2) #int division "//" would do floor

    return depth_counter, calcd_resulting

In [170]:
#hpyer parameters
label_col_name = "income"
num_trees_in_forest = 110

depth_threshold = 10 #32560 samples / 2 (10 times) = 32
# we could use a for loop to calc this based on sample size, and pass that to the two below 
split_leaf_minimum = 32

depth_threshold, split_leaf_minimum = sample_information(data.shape[0])

split_node_min_samples = (split_leaf_minimum * 2) + 1

In [171]:
print(depth_threshold, split_leaf_minimum, split_node_min_samples)

12 8.0 17.0


In [172]:
data.shape

(32560, 15)

In [173]:
data.head()
# Need to add column names


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


In [174]:
data.shape

(32560, 15)

In [175]:
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


In [176]:
test = pd.read_csv("adult/adult.test")


In [177]:
test.shape

(16281, 1)

In [178]:
test.head()

,,,,,,,,,,,,,,|1x3 Cross validator
25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K.


In [179]:
# does not work, not a csv names = pd.read_csv("adult/adult.names")

In [180]:
with open ("adult/adult.names", "r") as file:
    test = file.read()
print(test)

| This data was extracted from the census bureau database found at
| http://www.census.gov/ftp/pub/DES/www/welcome.html
| Donor: Ronny Kohavi and Barry Becker,
|        Data Mining and Visualization
|        Silicon Graphics.
|        e-mail: ronnyk@sgi.com for questions.
| Split into train-test using MLC++ GenCVFiles (2/3, 1/3 random).
| 48842 instances, mix of continuous and discrete    (train=32561, test=16281)
| 45222 if instances with unknown values are removed (train=30162, test=15060)
| Duplicate or conflicting instances : 6
| Class probabilities for adult.all file
| Probability for the label '>50K'  : 23.93% / 24.78% (without unknowns)
| Probability for the label '<=50K' : 76.07% / 75.22% (without unknowns)
|
| Extraction was done by Barry Becker from the 1994 Census database.  A set of
|   reasonably clean records was extracted using the following conditions:
|   ((AAGE>16) && (AGI>100) && (AFNLWGT>1)&& (HRSWK>0))
|
| Prediction task is to determine whether a person makes over

In [181]:
#"age", "workclass", "fnlwgt", "education", "education-num", "marital-status", "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss", "hours-per-week", "native-country", "income" or "class"???????

In [182]:
with open ("adult/index", "r") as file:
    test = file.read()
print(test)

Index of adult

02 Dec 1996      140 Index
10 Aug 1996  3974305 adult.data
10 Aug 1996     4267 adult.names
10 Aug 1996  2003153 adult.test



In [183]:
with open ("adult/adult.names", "r") as file:
    test = file.read()
print(test)

| This data was extracted from the census bureau database found at
| http://www.census.gov/ftp/pub/DES/www/welcome.html
| Donor: Ronny Kohavi and Barry Becker,
|        Data Mining and Visualization
|        Silicon Graphics.
|        e-mail: ronnyk@sgi.com for questions.
| Split into train-test using MLC++ GenCVFiles (2/3, 1/3 random).
| 48842 instances, mix of continuous and discrete    (train=32561, test=16281)
| 45222 if instances with unknown values are removed (train=30162, test=15060)
| Duplicate or conflicting instances : 6
| Class probabilities for adult.all file
| Probability for the label '>50K'  : 23.93% / 24.78% (without unknowns)
| Probability for the label '<=50K' : 76.07% / 75.22% (without unknowns)
|
| Extraction was done by Barry Becker from the 1994 Census database.  A set of
|   reasonably clean records was extracted using the following conditions:
|   ((AAGE>16) && (AGI>100) && (AFNLWGT>1)&& (HRSWK>0))
|
| Prediction task is to determine whether a person makes over

####Lab 1 Stuff

In [ ]:
# this is the split point function that we developed in class and used in assignment 1
# when we call it in our program we are iterating through a series of values to determine all splitpoints for the feature

def split_point_finder(x):
  mid_points_in_column = []
  x_sorted = sorted(x)

  for i in range(0, len(x)-1):
    mid_point = ( (x_sorted[i] + x_sorted[i+1]) / 2)
    mid_points_in_column.append(mid_point)
  return mid_points_in_column

In [ ]:
# this is the left and right node function we developed in class and used in assignment 1
# we use this to find the x-y pairs that will go to the left and ride nodes given a certain split point, we test all unique split points with this later

def branching_splits(x, y, split_point):
  mask = x >= split_point
  anti_mask = x < split_point
  x_right = x[mask]
  x_left = x[anti_mask]
  y_right = y[mask]
  y_left = y[anti_mask]
  return x_right, x_left, y_right, y_left

In [ ]:
# this function is pulled from assignment 1 and has a few slight modifications
# this functions purpose is to find the best split point and feature at each node of the decision tree
# it then returns the feature, split point, root node mse, branching leaf node weighted mse and the dictionary of all errors for the features tested

# what we have modified in this code is the ability to randomly select features at each node of the tree, a subset of the total training dataset features
# we do this by taking num_random_features_to_choose making it a parameter that can be adjusted in our decision tree creation function
# we specify the number of features we want to randomly choose, then before we begin looping for the feautures looking for the best feature / split point
# we randomly select from our total features in the dataset columns the parameter number of features to test for the splitting at this node
# so if we have a parameter of 5 and 17 feature columns in our dataframe, we randomly choose 5 of those 17 with duplicates not being allowed
# from there the function works the same way as assignment 1 where we determine the best split point and feature from those we test to determine the best splitting for this node

# we pass in the bootstrap data to this function not the total training dataset
def best_branching_random_features(data, label_column_name, num_random_features_to_choose):

  y = data[label_column_name]
  possible_features_to_test = data.drop(label_column_name, axis=1) # drop the label column from the dataframe so only features are left

  randomized_features_this_node = rng.choice(possible_features_to_test.columns, num_random_features_to_choose, replace = False)
  # we use rng.choice to randomly choose (parameter number of features) from (all feature columns that are present in the dataframe) and don't allow duplicates

  this_node_data = possible_features_to_test[randomized_features_this_node].copy() # then we create a new dataframe only copying the feature columns selected by our random feature selector above
  # we get a smaller dataframe with only the number of features declared in the function parameter

  feature_error_dictionary = {}

  # In assignment 1 we wouldn't have ran into these errors, but I ran into an error for each of these variables not being initialized
  # min error should be set to inf, anythign smaller will replace it
  minimum_error = float('inf')

  # these 3 can create errors further down our decision tree building if no split points are found to reduce the minimum error, if they arent initialized now and we go to return them
  # then we get the errors I pasted in below - if your testing a split with only has 100 samples and for a feature they are all the same value, then no split point exists and these
  # values would be left un-initialized given our structure below
  node_mse = float('inf')
  best_feature = 'Kelton'
  best_split_point_current_feature = 992018683
  #UnboundLocalError: cannot access local variable 'best_feature' where it is not associated with a value
  #UnboundLocalError: cannot access local variable 'node_mse' where it is not associated with a value
  #UnboundLocalError: cannot access local variable 'best_split_point_current_feature' where it is not associated with a value

  for feature in this_node_data.columns:
    x = data[feature]

    split_points = split_point_finder(x)
    split_points_unique = np.unique(split_points)

    for split_point in split_points_unique:
      x_right, x_left, y_right, y_left = branching_splits(x, y, split_point)
      unique_errors = weighted_branch_MSE(y,y_right,y_left)
      feature_error_dictionary[feature, split_point] = [unique_errors]

        # recording the best feature and split point based on lowest weighted branch mse from the mse function
      if unique_errors[1] < minimum_error:
        minimum_error = unique_errors[1]
        node_mse = unique_errors[0]
        best_feature = feature
        best_split_point_current_feature = split_point

  return feature_error_dictionary, node_mse, minimum_error, best_feature, best_split_point_current_feature


In [ ]:
# showing that the best_branching_random_features function delivers the best split point, feature, and mse of leaves along with root node mse
def main(data, label_column_name, num_randomized_features):
  total_samples = data.shape[0]

  dictionary_of_errors_for_node, root_mse, branch_mse, feature, split_point = best_branching_random_features(data, label_column_name, num_randomized_features)

  print(root_mse)
  print(branch_mse)
  print(feature)
  print(split_point)

  return dictionary_of_errors_for_node, root_mse, branch_mse, feature, split_point

In [ ]:
mcnair_data = data_sample_train.copy()
label_column_name = 'price'
number_of_random_features = int(np.sqrt((mcnair_data.shape[1])-1)) # rounds down to 4


dictionary_of_errors_for_node, root_mse, branch_mse, feature, split_point = main(bootstrap, label_column_name, number_of_random_features)

In [ ]:

features = []
for feature, split_point in dictionary_of_errors_for_node:
  features.append(feature)


In [ ]:
np.unique(features)

In [ ]:
# this is my decision tree builder function, which is similar to that of which was built in class but I went with a dictionary instead of a tuple return type
# I have set parameters for max depth of the tree, minimum number of samples in a node to try to split it, and a minimum number of samples in the resulting leaves to continue with the splitting


def dt_builder(bootstrap_current_node, label_column_name, num_randomized_features, minimum_samples_per_leaf, min_samples_per_split, max_depth, depth = 0):

  before_split_sample_size = bootstrap_current_node.shape[0]# get the number of samples in the current parent node


  # this is like the max depth checker in the in-class example, but we must also check to make sure that there are enough samples in the parent node to even determine if splitting is worthwhile
  if depth >= max_depth or before_split_sample_size < min_samples_per_split: # if depth is >= max depth or there arent enough samples in the parent node then we return a prediction
    return {"prediction_this_node": np.mean(bootstrap_current_node[label_column_name])}

  # we have not reached max depth or the minimum number of samples in the parent node, so we call our function to find the best splitting at this node
  # we pass thorugh the current node sample, the label, and the number of features to randomly choose
  dictionary_of_errors_for_node, root_mse, branch_mse, feature, split_point = best_branching_random_features(bootstrap_current_node, label_column_name, num_randomized_features)


  # this is where I began getting errors for the uninitialized best feature, split and root mse in our branching function
  # I have initialized the best feature to my name, if this is returned that means there were no valid split points found in the features randomly selected
  # this seems to be due to a small number of samples in the node and the sample feature values being the same, like all having 1.0 floors as an example, no splits found so kelton is returned
  # in the case that this non-feature basecase is returned we can no longer split and return this node as a prediction node
  if feature == 'Kelton':
    # print(feature) # testing
    return {"prediction_this_node": np.mean(bootstrap_current_node[label_column_name])}

  # just like branching splits we use vectorized operations on the current node to get the data for the resulting left and right leaf nodes from the split
  # I suppose you could store these in the error dictionary from the best_branching_random_features from above
  left_node_data = bootstrap_current_node[bootstrap_current_node[feature] < split_point]

  right_node_data = bootstrap_current_node[bootstrap_current_node[feature] >= split_point]

  # this gets the size of the resulting split leaf nodes, left and right, from the lead node data sets, we get the sizes to check the leaf node sample sizes against the minimum
  left_sample_size = left_node_data.shape[0]
  right_sample_size = right_node_data.shape[0]

  # here we are checking if both the left and right resulting leaf node samples from splitting are greater than or equal to the minimum value parameter we have set
  # if either is less than the parameter threshold we do not do the node split and we return the node as a prediction node instead
  if left_sample_size < minimum_samples_per_leaf or right_sample_size < minimum_samples_per_leaf:
    return {"prediction_this_node": np.mean(bootstrap_current_node[label_column_name])}

 # these are our recrusive calls to this function, for the left nodes and right nodes, each time we increase the depth by 1 so we know if we need to stop the branching due to depth
 # these calls are what create the tree leaf structure below the root node which is created the first time we call this function
  left_leaf_node = dt_builder(left_node_data, label_column_name, num_randomized_features, minimum_samples_per_leaf, min_samples_per_split, max_depth, depth + 1)
  right_leaf_node = dt_builder(right_node_data, label_column_name, num_randomized_features, minimum_samples_per_leaf, min_samples_per_split, max_depth, depth + 1)

  # just like our tuple return from the in class version of this code, we return the feature, the split point, the left leaf node and right leaf node, in dictionary form instead of tuples
  return { "feature": feature, "split_point": split_point, "left_leaf_node": left_leaf_node, "right_leaf_node": right_leaf_node}
  # I had dynamic depth tracking in the key of the leaf nodes, but that was making tree navigation more difficult so I opted to remove it

In [ ]:
def predict_traversal(individual_tree, test_data):
# in the in class version with tuples and different data format, we can check if isinstance tree,float - which wouldn't work for multiple reasons in this assignment
# I have opted for the decision tree to be a dictionary format instead of a tuple, both are hard to read though
# instead of checking for a float, because I have set a key for when a node becomes a prediciton, we check for the key "prediction_this_node" each time we call this function, and when it finds this key it returns
# the prediction value that is associated with that node
  if "prediction_this_node" in individual_tree:
    return individual_tree["prediction_this_node"]

  # we need to pull the feature and split point values from each dictionary entry to traverse further down, once we have the feature and the split we can test the value of the feature of the sample datapoint against the split point
  # so we use the keys to get the values
  current_feature = individual_tree["feature"]
  current_split_point = individual_tree["split_point"]


  # if the value for this samples feature value is less than the split point we traverse to the left child node
  # if the value for this samples feature value is greater than or equal to the split point then we traverse to the right child node
  # this traversal will come to an end when we encounter the key indicating a prediciton node
  if test_data[current_feature] < current_split_point:
    return predict_traversal(individual_tree["left_leaf_node"], test_data) # go to the next left node
  else:
    return predict_traversal(individual_tree["right_leaf_node"], test_data) # go to the next right node

In [ ]:
# creating a single test decision tree, min samples, min samples to even try to split is at least 2x the minimum samples per leaf
# I was trying to experiment with setting the min samples to consider splitting higher than 2x the min samples per leaf
# max depth of 10, https://www.geeksforgeeks.org/how-to-choose-ideal-decision-tree-depth-without-overfitting/
# in the link they are using a classifier model, so this may be irrelevant to a regression model
# but they test depths from 1-15 and their accuraccy seems to almost peak around a max depth of 7
# I have been testing between 3-12 and I seem to be getting the best results around 8-12 given the parameter of 4 randomly chosen features I have been using,
mcnair_data = data_sample_train.copy()
label_column_name = 'price'
num_randomized_features = int(np.sqrt((mcnair_data.shape[1]-1)))
minimum_samples_per_leaf = 12
min_samples_per_split = (minimum_samples_per_leaf * 2) + 1
max_depth = 10
bootstrap_sample_size = mcnair_data.shape[0]

In [ ]:
    X_bootstrap, y_bootstrap, X_oob, y_oob, bootstrap_current_node, all_oob = boot_strap_and_oob(data, label_column_name, bootstrap_sample_size)
    individual_tree_dict = dt_builder(bootstrap_current_node, label_column_name, num_randomized_features, minimum_samples_per_leaf, min_samples_per_split, max_depth, depth = 0)
    individual_tree_dict

####Lab 2 Stuff

In [184]:
# In this function I am taking in the training dataset, the label and the total number of train samples
# for each training sample in the dataset we randomly grab a row from the training dataset to create our bootstrap dataset
# so in our case we have a training dataset of 8500, so we create a bootstrap that has 8500 samples from our original train dataset
# we get indices to pull from the training set randomly by generating a random # from 0 to 8499 and adding that row to our array
# once we have created our array of samples from the rows of the training dataset, we create the bootstrap dataset from our array of rows
# this method will allow duplicates, and with this many samples we should end up with roughly 65% of the train dataset in our bootstrap
# then we seperate the label column from the X_bootstrap resulting in just the training data with no label
# then y_bootstrap is just the labels column, but all of the indices match up

def bootstrap_creator(this_tree_data, label_column_name, total_samples):
  bootstrap_for_this_tree = [] # holder array

  for i in range(total_samples): # match size of bootstrap to size of training data set
    random_index_to_add = np.random.randint(0 , total_samples) # generate a row index to grab from the training datatset
    bootstrap_for_this_tree.append(this_tree_data.iloc[random_index_to_add]) # add that dataset row to our array


    #create the bootstrap dataframe
  bootstrap = pd.DataFrame(bootstrap_for_this_tree) # I suppose I could have np.unique before creating the dataframe here - nevermind that didn't work how I intended

    # generate X and Y bootstraps from bootstrap
  X_bootstrap = bootstrap.drop(label_column_name, axis=1)
  y_bootstrap = bootstrap[label_column_name]

  return X_bootstrap, y_bootstrap, bootstrap

In [185]:
# this function is used to calculate the rows from the training dataset that did not make it into this bootstrap dataset
# we pass in the label, the training dataset and the randomly selected bootstrap dataset
# with a dataframe this is a bit simpler than with the all integer numpy array from the class example
# here we can simply drop all of the indices (rows) that exist in the bootstrap data set from the training data set and we are left with the out of box sample set
# I didn't need to copy the dataframe, I find myself doing this somewhat often when I am working with dataframes to maintain the last version of them
# I want to say I picked up this habit during my AI course but I don't remember why exactly, I think it had to do with model evaluations
# then to find the data frames for X_oob and y_oob we simply drop the label from our total oob dataframe for X and pull out only the label column for our y

def oob_determination(tree_data, bootstrap, label_column_name):

    # drop bootstrap rows from the total tree training data set to be left with a dataframe of only the none-selected training samples
  oob_for_this_tree = tree_data.drop(index=bootstrap.index) # drop all the indices from the tree dataset/ training dataset that exist within the bootstrap for this specific tree

    # copy oob_for_this_tree incase we need to pass it as a return later
  all_oob_this_tree = oob_for_this_tree.copy()

    # make the X-oob and y_oob respective dataframes
  X_oob = oob_for_this_tree.drop(label_column_name, axis=1)
  y_oob = oob_for_this_tree[label_column_name]

  return X_oob, y_oob, all_oob_this_tree


In [186]:
#this function calls the bootstrap creation function and then passes the bootstrap dataset into oob to return the oob values
# meaning bootstrap + oob below make up the entirety of the 8500 data samples within sample_data_train without any overlap
# and this function just returns the output of both lumped together
def boot_strap_and_oob(tree_data, label_column_name, total_samples):
  X_bootstrap, y_bootstrap, bootstrap = bootstrap_creator(tree_data, label_column_name, total_samples)
  X_oob, y_oob, all_oob = oob_determination(tree_data, bootstrap, label_column_name)

  return X_bootstrap, y_bootstrap, X_oob, y_oob, bootstrap, all_oob


In [187]:
# demonstration of these functions working correctly, passing in the the training sample number of samples and retreiving all of the bootstrap & oob data frames
def main_test_bs(data, label_column_name):
  total_samples = data.shape[0]
  X_bootstrap, y_bootstrap, X_oob, y_oob, bootstrap, all_oob = boot_strap_and_oob(data, label_column_name, total_samples)
  return X_bootstrap, y_bootstrap, X_oob, y_oob, bootstrap, all_oob

In [188]:
mcnair_data = data.copy()
label_column_name = 'income'

X_bootstrap, y_bootstrap, X_oob, y_oob, bootstrap, all_oob = main_test_bs(mcnair_data, label_column_name)


In [189]:
bootstrap.drop_duplicates().sort_index()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
5,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32549,43,Self-emp-not-inc,27242,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,50,United-States,<=50K
32552,32,Private,116138,Masters,14,Never-married,Tech-support,Not-in-family,Asian-Pac-Islander,Male,0,0,11,Taiwan,<=50K
32554,22,Private,310152,Some-college,10,Never-married,Protective-serv,Not-in-family,White,Male,0,0,40,United-States,<=50K
32555,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K


In [190]:
all_oob.drop_duplicates().sort_index()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
9,37,Private,280464,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,Black,Male,0,0,80,United-States,>50K
13,40,Private,121772,Assoc-voc,11,Married-civ-spouse,Craft-repair,Husband,Asian-Pac-Islander,Male,0,0,40,?,>50K
15,25,Self-emp-not-inc,176756,HS-grad,9,Never-married,Farming-fishing,Own-child,White,Male,0,0,35,United-States,<=50K
30,20,Private,266015,Some-college,10,Never-married,Sales,Own-child,Black,Male,0,0,44,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32551,43,Private,84661,Assoc-voc,11,Married-civ-spouse,Sales,Husband,White,Male,0,0,45,United-States,<=50K
32553,53,Private,321865,Masters,14,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,>50K
32556,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32557,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K


Working on getting multiprocessing setup

In [191]:
from multiprocessing import Pool # multiprocessing but requires function to be in a .py file when working with jupyter?
from multiprocessing import get_context #older fork method not newer spawn, issues with this also, need to put functions in .py file maybe

# https://docs.python.org/3.10/library/multiprocessing.html

L